In [1]:
from petrelic.multiplicative.pairing import G1, G2, GT
from credential import *
import pickle
import hashlib

In [2]:
# Pedersen commitment

# G cyclic group of order p
p = G1.order()
g = G1.generator()
h = G1.generator()

In [3]:
# Prover input
x = G1.order().random()
r = G1.order().random()

In [4]:
# Commitment
C = g ** x * h ** r

In [5]:
# Zero knowledge proof
# Prover
# 1. Choose random r_x, r_r
r_x = G1.order().random()
r_r = G1.order().random()

In [6]:
# 2. Compute commitment
R = g ** r_x * h ** r_r
R

G1Element(03116d010e5303107b603facd6d4b61ec86d9f9b6283a5acf3110b0ffef5ede77f9ec37715b9141fcdbf4b34d4d581a903)

In [7]:
def get_challenge(public_values, p):
    c = hashlib.sha256()
    for value in public_values:
        c.update(value.to_binary())
    return bytes_to_Z_p(c.digest(), p)

In [8]:
# 3. Compute challenge
c = get_challenge([g, h, C, R], p)

TypeError: bytes_to_Z_p() takes 1 positional argument but 2 were given

In [ ]:
s_x = (r_x - c * x) % p
s_r = (r_r - c * r) % p

In [9]:
R == C ** c * g ** s_x * h ** s_r

NameError: name 'c' is not defined

In [10]:
public_keys = ...
prover_input = ...

In [11]:
def get_commitment(public_keys):
    random_values = [G1.order().random() for _ in public_keys]
    R = 1
    for key, value in zip(public_keys, random_values):
        R *= key ** value
    return random_values, R

In [12]:
# i, attr (Bn)
# t (Bn)

def get_C(pk, user_attributes):
    # pick random t from integers modulo p
    t = G1.order().random()
    
    # compute commitment
    C = pk.g ** t
    for (i, attr) in user_attributes.get_attributes():
        C *= pk.Y[i] ** bytes_to_Z_p(attr, G1.order())
    return C

In [13]:
def get_commitment(pk, user_attributes):
    randoms = [G1.order().random() for _ in user_attributes.get_attributes() + 1]

    R = pk.g ** randoms[0]

    for (i, attr), random in zip(user_attributes.get_attributes(), randoms[1:]):
        R *= pk.Y[i] ** random
    
    return randoms, R

In [14]:
def get_challenge(public_values):
    c = hashlib.sha256()
    for value in public_values:
        c.update(value.to_binary())
    return bytes_to_Z_p(c.digest(), G1.order())

In [15]:
def get_response(randoms, user_attributes, t, challenge):
    responses = [random[0] - challenge*t]
    for random, (i, attr) in zip(randoms[1:], user_attributes.get_attributes()):
        responses.append((random - challenge * bytes_to_Z_p(attr, G1.order())) % G1.order())
    return responses

In [16]:
# prover input
# t, attr (Bn)

# generators
generators = [pk.g] + [pk.Y[i] for i, _ in user_attributes.get_attributes()]

# prover input
# t, attr (Bn)
prover_inputs = [t] + [bytes_to_Zp(attr) for _, attr in user_attributes.get_attributes()]

NameError: name 'pk' is not defined

In [17]:
sk, pk = generate_key(10*[1])
pk.g

G1Element(0217f1d3a73197d7942695638c4fa9ac0fc3688c4f9774b905a14e3a3f171bac586c55e83ff97a1aeffb3af00adb22c6bb)

In [18]:
pk.Y

[G1Element(030884d691e3b72376f9e9ccb37e0cf38103fdddbcbd8ac6b74c134c5eb79f06e870ea0cdca575557a532e45280e93100b),
 G1Element(0316c3a5e578890886b4b0bf741d15d933ffa6302238f970bdd9a35d3c5a55b6dd3fb7417baa93ee47087a43f410e3f52d),
 G1Element(02004779ee53d59142889804d2875ae4f2e3b9bd412330979a2a67aedf22273d8d2b6f0c8b9aa80f047159ed289602b4a9),
 G1Element(03015fbcd0ec85a8e510bdd565ca4d56ae63422a13159df27c1ccc7b3edd009f326e2a7e50eb5d8b5f87ed00c2a0fa32d8),
 G1Element(030a880a4a2a3e851509f20d19af0acc4d9581f291b4603ae5fe55b24aa97f5242cffb469741c73b0ef8dd0e26837646c3),
 G1Element(0212d71ce0fbd2e0e8ef5dcc7a33606c3d714790cfb267cb2bf9200b6bcd1478f73d9f0f702c71a5f14d31ea3358c502ae),
 G1Element(020ab5c89fcdc02b8f0c272de786e6fa0bd357f8fd698bfd4d567c546b3ebd835f215a7bb82944139b8fce7a03820e194d),
 G1Element(020d561ba2cd732aa9f505fa65ac3089bc6679b3454b1362daa5e156ce61e3b308a1e29ef5f251e8eb52599b0efba6b8b3),
 G1Element(03130b0d0ff20c801dc65ae325e111f7a4b16bede5d471434e3e64b66afe7ba8af3c43d27f226199bfca8626db477

In [19]:
from petrelic.multiplicative.pairing import G1Element

def get_zkp_commitment(
        generators : List[G1Element]
        ) -> Tuple[G1Element, List[Bn]]:
    """ Generate a commitment """
    # pick a list of random numbers from integers modulo p
    randoms = [G1.order().random() for _ in generators]

    # compute commitment
    R = generators[0] ** randoms[0]
    for generator, random in zip(generators[1:], randoms[1:]):
        R *= generator ** random

    return randoms, R

In [20]:
def get_zkp_challenge(
        generators : List[G1Element],
        com : Bn,
        R : G1Element
        ) -> Bn:
        """ Generate a non-interactive challenge according to the Fiat-Shamir heuristic """
        c = hashlib.sha256()
        for generator in generators:
                c.update(generator.to_binary())
        c.update(com.to_binary())
        c.update(R.to_binary())
        return bytes_to_Z_p(c.digest())

In [21]:
def get_zkp_response(
        randoms: List[Bn],
        c: Bn,
        prover_input: List[Bn]
        ) -> List[Bn]:
        """ Generate a response """
        return [(random - c * input).mod(G1.order()) for random, input in zip(randoms, prover_input)]

In [22]:
def verify_zkp(
        R: G1Element,
        com: G1Element,
        c: Bn,
        generators: List[G1Element],
        response: List[Bn]
        ) -> bool:
        """ Verify a zero-knowledge proof """
        lhs = R
        rhs = com**c
        for generator, resp in zip(generators, response):
                rhs *= generator ** resp
        return lhs == rhs

In [23]:
L = 5
sk, pk = generate_key([b"0"] * L)

user_attributes = AttributeMap(L)
user_attributes.set_attribute(0, b'hello')
user_attributes.set_attribute(1, b'1')
user_attributes.set_attribute(2, b'2')

In [24]:
# pick random t from integers modulo p
t = G1.order().random()

# compute commitment
C = pk.g ** t
for (i, attr) in user_attributes.get_attributes():
    C *= pk.Y[i] ** bytes_to_Z_p(attr)

In [25]:
C

G1Element(02035d3b239cb09667a340f808f7095a289afed1007e4cd13618574d5af05791bec29ad38290ae22345b3b435d6154c716)

In [26]:
# prover input
# t, attr (Bn)

# generators
generators = [pk.g] + [pk.Y[i] for i, _ in user_attributes.get_attributes()]

# prover input
# t, attr (Bn)
prover_inputs = [t] + [bytes_to_Z_p(attr) for _, attr in user_attributes.get_attributes()]

In [27]:
generators

[G1Element(0217f1d3a73197d7942695638c4fa9ac0fc3688c4f9774b905a14e3a3f171bac586c55e83ff97a1aeffb3af00adb22c6bb),
 G1Element(030f83ee2c00684abac384c477a8173a3a4b5821560b79ab72445707f15d03e555916384f1499de1674379f4a2d575d2f2),
 G1Element(030a1f7f79367a78cb34a4d21f3886224d2b9d9fd58fa8bc2b5224cfb946456ecbf1def1948c71b1f1e94272fd9191cac8),
 G1Element(020bab85821fda41c6bf8ff97b3b68d8c6c43c1377a5c53a07c464ebb1aabccceca9f9edf54ff9ce5f17a002e6e73c2309)]

In [28]:
prover_inputs

[Bn(35846693713698439960853117498377448791333903644787183196605534079416094967898),
 Bn(20329878786436204988385760252021328656300425018755239228739303522659023427620),
 Bn(48635463943209834798109814161294753926839975257569795305637098542720658922315),
 Bn(43658286468849876353920127463240192620539495994902638395191669966194749975348)]

In [29]:
randoms, R = get_zkp_commitment(generators)
print("Randoms: ", randoms)
print("R: ", R)

Randoms:  [Bn(36652827464143185810952154712574822203782711103348179358491341243715408016344), Bn(35234697353464543378943975766439153736313695384551062629354656258975319570643), Bn(22419324642539633055219523681734339888843564254822813265399973234028462191139), Bn(47353100529918075922924061726769184450943495750859026380862800655882026978667)]
R:  G1Element(0215d14002543dd81bf15e1722f220b3f02b2b6c275a5589685c0665cbcd071be3a7cb30b37be5717d033bc47694383f02)


In [30]:
c = get_zkp_challenge(generators, C, R)
print("Challenge: ", c)

Challenge:  24815522912930042130269440612313777510725396122789907962427973435978058493570


In [31]:
c

Bn(24815522912930042130269440612313777510725396122789907962427973435978058493570)

In [32]:
response = get_zkp_response(randoms, c, prover_inputs)
print("Response: ", response)
# response[3] = Bn(2)

Response:  [Bn(13977764089080994701520257059824533322228551292855587099447807569204051334230), Bn(51374018868899559629132916644760406486383470798674238979850584892422052586873), Bn(31602149107291604980823903207394530214316926919018607085665885256543340516707), Bn(44705627686242831139595078441366391176688923986096009979498370113729211549717)]


In [33]:
verify_zkp(R, C, c, generators, response)

True